In [49]:
import os
os.chdir( os.getenv('HOME') + '/git/orakel')
from importlib import reload
import re

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

import numpy as np
import pandas as pd
from numpy import fft

from micropred import mp_utils as mpu


Loading BokehJS ...

In [5]:
sponsors = mpu.sponsors()
all_electricity_streams = sponsors['Offcast Goose']
# %%
load_streams = [ stream for stream in all_electricity_streams
                 if re.match( 'elect.*-load-*', stream ) ]
load_streams

['electricity-load-nyiso-capitl.json',
 'electricity-load-nyiso-centrl.json',
 'electricity-load-nyiso-dunwod.json',
 'electricity-load-nyiso-genese.json',
 'electricity-load-nyiso-hud_valley.json',
 'electricity-load-nyiso-longil.json',
 'electricity-load-nyiso-mhk_valley.json',
 'electricity-load-nyiso-millwd.json',
 'electricity-load-nyiso-north.json',
 'electricity-load-nyiso-nyc.json',
 'electricity-load-nyiso-overall.json',
 'electricity-load-nyiso-west.json']

In [114]:
N = 12 * 24 * 3 
N

864

In [282]:
stream_df = mpu.lagged( 'electricity-load-nyiso-overall.json', count=1000).sort_values('epoch_time')
stream_df['time'] = stream_df['epoch_time'] - stream_df['epoch_time'].iloc[0]

In [159]:
stream_df.head()

,epoch_time,value,tstamp_utc
1000,1.603328e+09,17078.3041,2020-10-22 00:55:53.938965321
999,1.603328e+09,17014.7481,2020-10-22 01:00:53.325997829
998,1.603329e+09,16958.8264,2020-10-22 01:05:52.808196068
997,1.603329e+09,16881.2229,2020-10-22 01:10:53.083670855
996,1.603329e+09,16795.7317,2020-10-22 01:15:53.298027039


In [283]:
# create a new plot (with a title) using figure
p = figure(plot_width=950, plot_height=400, title="My Line Plot")
# add a line renderer
p.line( stream_df['tstamp_utc'], stream_df['value'], line_width=2)
show(p) # show the results

In [149]:
stream_df.shape

(1001, 3)

In [284]:
N = 12 * 24 * 3  # 3 days of once every 5 minute data
Nf = 20
i = np.linspace( 0, N - 1, N + 1)

In [132]:
len(i)

864

In [304]:
feats = np.array(   [ np.cos( 2 * np.pi * k * i / N ) for k in range(0, Nf + 1) ] 
                  + [ np.sin( 2 * np.pi * k * i / N ) for k in range(-Nf, Nf + 1) if k !=0 ] 
                  + [ i / N ]
                  + [ (i / N)**2 ]
                ).transpose()

In [109]:
from sklearn.linear_model import LinearRegression

In [305]:
lr = LinearRegression()

In [129]:
s = 0 

In [306]:
lr.fit( feats[:s+N,:], stream_df['value'][s: s+N])

LinearRegression()

In [307]:
stream_df['preds'] = float('nan')
stream_df.loc[ stream_df.index.values[:len(feats)], 'preds'] = lr.predict( feats ) 

print( len(feats))

865


In [308]:
from statsmodels.tsa.arima.model import ARIMA

stream_df['error'] = stream_df['value'] - stream_df['preds']

err = stream_df['error']

arima = ARIMA( list( err[err.notnull()] ) , order=(2,0,1))

model_fit = arima.fit()
# arima.predict()

In [298]:
model_fit.mle_retvals

{'fopt': 5.049056726576018,
 'gopt': array([-3.10023118e-05,  3.08002512e-06, -2.78362222e-05, -1.10177645e-05,
         6.80349110e-05]),
 'fcalls': 72,
 'warnflag': 0,
 'converged': True,
 'iterations': 10}

In [299]:
model_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  865
Model:                 ARIMA(2, 0, 1)   Log Likelihood               -4367.434
Date:                Mon, 26 Oct 2020   AIC                           8744.868
Time:                        20:08:00   BIC                           8768.682
Sample:                             0   HQIC                          8753.982
                                - 865                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1353      3.517      0.038      0.969      -6.758       7.029
ar.L1          0.6987      0.284      2.461      0.014       0.142       1.255
ar.L2          0.0086      0.167      0.051      0.959      -0.319       0.337
ma.L1         -0.2060      0.279     -0.738      0.461      -0.753       0.341
sigma2      1425.5616     66.368     21.480      0.000    1295.482    1555.641
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):                 2.31
Prob(Q):                              1.00   Prob(JB):                         0.31
Heteroskedasticity (H):               1.07   Skew:                             0.08
Prob(H) (two-sided):                  0.55   Kurtosis:                         3.20
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [309]:
stream_df.loc[  stream_df.index[:len(feats)], 'err_pred_0'] = model_fit.predict()

In [310]:
stream_df[ 'err_pred_1'] = stream_df['err_pred_0'].shift(1)

In [293]:
stream_df.head(50)

,epoch_time,value,tstamp_utc,time,preds,error,err_pred_0,err_pred_1
1000,1.603460e+09,17020.5167,2020-10-23 13:25:52.701493979,0.000000,16985.368250,35.148450,0.130073,NaN
999,1.603460e+09,17030.3974,2020-10-23 13:30:52.972314835,300.270821,17002.389280,28.008120,19.940721,0.130073
998,1.603460e+09,17034.7096,2020-10-23 13:35:52.792781591,600.091288,17018.176130,16.533470,18.247778,19.940721
997,1.603460e+09,17024.0978,2020-10-23 13:40:52.812279701,900.110786,17032.785673,-8.687873,11.677215,18.247778
996,1.603461e+09,17004.7051,2020-10-23 13:45:52.838097811,1200.136604,17046.262861,-41.557761,-2.250461,11.677215
995,1.603461e+09,17047.2866,2020-10-23 13:50:52.850018978,1500.148525,17058.642423,-11.355823,-21.653481,-2.250461
994,1.603461e+09,17047.2866,2020-10-23 13:55:53.439202547,1800.737709,17069.949552,-22.662952,-10.657894,-21.653481
993,1.603462e+09,17075.5964,2020-10-23 14:00:52.871877909,2100.170384,17080.202680,-4.606280,-12.592702,-10.657894
992,1.603462e+09,17151.1554,2020-10-23 14:05:53.544645309,2400.843151,17089.413557,61.741843,-5.001505,-12.592702
991,1.603462e+09,17046.6054,2020-10-23 14:10:52.606600046,2699.905106,17097.590299,-50.984899,30.158430,-5.001505


In [311]:
stream_df['pred1'] = stream_df['preds'] + stream_df['err_pred_0']

In [244]:
from numpy.linalg import norm

In [312]:
norm( (stream_df['value'] - stream_df['preds'])[stream_df.preds.notnull()], 2)/ np.sqrt(len(feats)) ,\
norm( (stream_df['value'] - stream_df['preds'])[stream_df.preds.notnull()], 1)/ len(feats), \
(stream_df['value'] - stream_df['pred1']).abs().mean()

(47.27479617896492, 37.36251421965314, 29.97562007466205)

In [279]:
err.notnull().sum() ,len( model_fit.predict() )

stream_df['err2'] = float("nan") 
stream_df.loc[ stream_df.index[:len(feats)], 'err2' ] = err[err.notnull()] -  model_fit.predict()

In [180]:
# create a new plot (with a title) using figure
p = figure(plot_width=950, plot_height=400, title="My Line Plot")
# add a line renderer
p.line( stream_df['tstamp_utc'], stream_df['value'], line_width=2)
p.line( stream_df['tstamp_utc'], stream_df['preds'], line_width=2, line_color='red')
show(p) # show the results

In [280]:
stream_df['error'].abs().mean(), stream_df['err2'].abs().mean() 

(36.84638787760053, 30.368963804046547)

In [216]:
stream_df['error'] = stream_df['value'] - stream_df['preds']
# create a new plot (with a title) using figure
p = figure(plot_width=950, plot_height=400, title="My Line Plot")
# add a line renderer
# p.line( stream_df.index.values, stream_df['error'], line_width=2)
p.line( stream_df.index.values, stream_df['err2'], line_width=2, color='red')

show(p) # show the results

In [136]:
pred = lr.predict( feats[[s+N],:]) 

In [138]:
pred, stream_df['value'][s+N]

(array([15282.28406222]), 15993.876799999998)

In [76]:
fft_raw = fft.fft( stream_df['value'])


In [77]:
def np_norm( x: np.array ):
    """For x an array of complex number return elementwise norms"""
    return np.sqrt( np.real( x * np.conj(x) ))
    

In [78]:
fft_df = pd.DataFrame( {'fft': fft_raw, 'w': list( range(len(fft_raw)) ) } )

fft_df['norm'] = np_norm( fft_df['fft'] * fft_df['fft'] )

fft_df

,fft,w,norm
0,1.565631e+07+0.000000e+00j,0,2.451200e+14
1,-3.153846e+05+1.262803e+05j,1,1.154141e+11
2,-2.694118e+05+2.271744e+05j,2,1.241909e+11
3,-5.494853e+05+6.737096e+05j,3,7.558188e+11
4,3.246161e+05-5.378029e+05j,4,3.946076e+11
...,...,...,...
996,1.089641e+05+1.245606e+05j,996,2.738852e+10
997,3.246161e+05+5.378029e+05j,997,3.946076e+11
998,-5.494853e+05-6.737096e+05j,998,7.558188e+11
999,-2.694118e+05-2.271744e+05j,999,1.241909e+11


In [79]:
w = fft_df['w']
chart = alt.Chart( fft_df.drop( columns=['fft'])[ (w > 0) & (w < 100)] )
chart.mark_bar().encode( x='w', y='norm' )

alt.Chart(...)

In [83]:
fft_filt = fft_raw.copy()
fft_filt[30:] = 0.0
smooth = fft.ifft( fft_filt )

In [84]:
p = figure(plot_width=950, plot_height=400, title="My Line Plot")
p.scatter( range(1,30), np_norm( fft_filt[1:30] )) 
show(p) # show the results

In [55]:
stream_df['smooth'] = np.real( smooth )

In [58]:
p = figure(plot_width=950, plot_height=400, title="My Line Plot")
p.line( stream_df['tstamp_utc'], stream_df['value'], line_width=2)
p.line( stream_df['tstamp_utc'], stream_df['smooth'], line_width=2)
show(p) # show the results